# PCA-Analyse der Schweizer Abstimmungsdaten

Diese Analyse untersucht die politischen Dimensionen im Abstimmungsverhalten der Schweizer Gemeinden.

**Basierend auf:** Hermann & Leuthold - Der politische Raum der Schweiz

**Die drei Dimensionen:**
1. **PC1: Liberal ↔ Konservativ** (gesellschaftlich: Einbürgerung, EU, Ausländerpolitik)
2. **PC2: Establishment ↔ Populistisch** (Regierungstreue vs. populistische Initiativen)
3. **PC3: Ökologisch ↔ Technokratisch** (Umwelt, Energie, Pestizide)

**Wichtiger Hinweis zu PC2:**
Die wirtschaftliche Links-Rechts-Dimension ist nicht eindeutig von PC2 abgebildet. 
PC2 trennt primär zwischen:
- **Establishment** (Goldküste: folgt Bundesratsempfehlungen bei AHV-Reformen)
- **Populistisch** (Innerschweiz: gegen Rentenaltererhöhung, für populistische Initiativen)

Dies erklärt, warum die konservative Innerschweiz (SVP) auf PC2 ähnlich wie linke Gemeinden erscheint.

---

## 1. Setup und Daten laden

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Plot-Einstellungen
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

# Datenbankverbindung
DB_PATH = '../data/processed/swiss_votings.db'
conn = sqlite3.connect(DB_PATH)

print("Verbindung zur Datenbank hergestellt.")

In [ ]:
# Abstimmungsdaten laden
query = """
SELECT 
    municipality_id,
    municipality_name,
    proposal_id,
    voting_date,
    title_de,
    ja_prozent
FROM v_voting_results_analysis
WHERE ja_prozent IS NOT NULL
ORDER BY municipality_id, voting_date, proposal_id
"""

df_raw = pd.read_sql_query(query, conn)
print(f"Rohdaten geladen: {len(df_raw):,} Zeilen")
print(f"Gemeinden: {df_raw['municipality_id'].nunique()}")
print(f"Abstimmungsvorlagen: {df_raw['proposal_id'].nunique()}")
df_raw.head()

## 2. Datenmatrix erstellen (Gemeinden × Vorlagen)

In [ ]:
# Pivot: Zeilen = Gemeinden, Spalten = Abstimmungsvorlagen
# Werte = Ja-Prozent

df_pivot = df_raw.pivot_table(
    index=['municipality_id', 'municipality_name'],
    columns='proposal_id',
    values='ja_prozent',
    aggfunc='first'  # Sollte keine Duplikate geben
)

print(f"Matrix-Dimensionen: {df_pivot.shape[0]} Gemeinden × {df_pivot.shape[1]} Vorlagen")
print(f"\nFehlende Werte pro Vorlage (erste 10):")
print(df_pivot.isnull().sum().head(10))

In [ ]:
# Fehlende Werte analysieren
missing_per_municipality = df_pivot.isnull().sum(axis=1)
missing_per_proposal = df_pivot.isnull().sum(axis=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Fehlende Werte pro Gemeinde
axes[0].hist(missing_per_municipality, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Anzahl fehlender Vorlagen')
axes[0].set_ylabel('Anzahl Gemeinden')
axes[0].set_title('Fehlende Werte pro Gemeinde')
axes[0].axvline(missing_per_municipality.median(), color='red', linestyle='--', 
                label=f'Median: {missing_per_municipality.median():.0f}')
axes[0].legend()

# Fehlende Werte pro Vorlage
axes[1].hist(missing_per_proposal, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Anzahl fehlender Gemeinden')
axes[1].set_ylabel('Anzahl Vorlagen')
axes[1].set_title('Fehlende Werte pro Vorlage')
axes[1].axvline(missing_per_proposal.median(), color='red', linestyle='--',
                label=f'Median: {missing_per_proposal.median():.0f}')
axes[1].legend()

plt.tight_layout()
plt.savefig('missing_values_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nGemeinden mit >50% fehlenden Werten: {(missing_per_municipality > df_pivot.shape[1]/2).sum()}")
print(f"Vorlagen mit >50% fehlenden Gemeinden: {(missing_per_proposal > df_pivot.shape[0]/2).sum()}")

In [ ]:
# Entscheidung: Gemeinden und Vorlagen mit zu vielen fehlenden Werten entfernen
# Dann restliche NaN mit Spalten-Mittelwert imputieren

# Threshold: Max 20% fehlende Werte
MAX_MISSING_RATIO = 0.20

# Vorlagen filtern
valid_proposals = missing_per_proposal[missing_per_proposal <= df_pivot.shape[0] * MAX_MISSING_RATIO].index
df_filtered = df_pivot[valid_proposals]
print(f"Nach Vorlagen-Filter: {df_filtered.shape[1]} Vorlagen behalten (von {df_pivot.shape[1]})")

# Gemeinden filtern
missing_per_municipality_filtered = df_filtered.isnull().sum(axis=1)
valid_municipalities = missing_per_municipality_filtered[missing_per_municipality_filtered <= df_filtered.shape[1] * MAX_MISSING_RATIO].index
df_filtered = df_filtered.loc[valid_municipalities]
print(f"Nach Gemeinden-Filter: {df_filtered.shape[0]} Gemeinden behalten (von {df_pivot.shape[0]})")

print(f"\nFinale Matrix: {df_filtered.shape[0]} × {df_filtered.shape[1]}")
print(f"Verbleibende NaN: {df_filtered.isnull().sum().sum()} ({100*df_filtered.isnull().sum().sum()/(df_filtered.shape[0]*df_filtered.shape[1]):.2f}%)")

In [ ]:
# Imputation: Fehlende Werte mit Spalten-Mittelwert (= Durchschnitt der Vorlage) füllen
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(df_filtered)

# DataFrame für spätere Referenz
df_imputed = pd.DataFrame(
    X_imputed, 
    index=df_filtered.index, 
    columns=df_filtered.columns
)

print(f"Imputation abgeschlossen. Shape: {df_imputed.shape}")
print(f"Verbleibende NaN: {df_imputed.isnull().sum().sum()}")

## 3. Standardisierung

In [ ]:
# Z-Standardisierung (wichtig für PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_imputed)

print(f"Standardisierte Matrix: {X_scaled.shape}")
print(f"Mittelwert (sollte ~0 sein): {X_scaled.mean():.6f}")
print(f"Std (sollte ~1 sein): {X_scaled.std():.6f}")

## 4. Bestimmung der optimalen Faktorenzahl

In [ ]:
# Vollständige PCA für Scree-Plot
pca_full = PCA()
pca_full.fit(X_scaled)

# Eigenwerte und erklärte Varianz
eigenvalues = pca_full.explained_variance_
explained_variance_ratio = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

print("Erste 10 Komponenten:")
print("-" * 60)
print(f"{'PC':>4} {'Eigenwert':>12} {'Varianz %':>12} {'Kumulativ %':>12}")
print("-" * 60)
for i in range(10):
    print(f"{i+1:>4} {eigenvalues[i]:>12.3f} {explained_variance_ratio[i]*100:>11.2f}% {cumulative_variance[i]*100:>11.2f}%")

In [ ]:
# Scree-Plot und kumulative Varianz
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree-Plot (erste 20 Komponenten)
n_show = 20
axes[0].plot(range(1, n_show+1), eigenvalues[:n_show], 'bo-', markersize=8)
axes[0].axhline(y=1, color='r', linestyle='--', label='Kaiser-Kriterium (Eigenwert = 1)')
axes[0].set_xlabel('Hauptkomponente')
axes[0].set_ylabel('Eigenwert')
axes[0].set_title('Scree-Plot')
axes[0].set_xticks(range(1, n_show+1))
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Kumulative erklärte Varianz
axes[1].plot(range(1, n_show+1), cumulative_variance[:n_show]*100, 'go-', markersize=8)
axes[1].axhline(y=70, color='orange', linestyle='--', label='70% Varianz')
axes[1].axhline(y=80, color='red', linestyle='--', label='80% Varianz')
axes[1].set_xlabel('Anzahl Hauptkomponenten')
axes[1].set_ylabel('Kumulative erklärte Varianz (%)')
axes[1].set_title('Kumulative erklärte Varianz')
axes[1].set_xticks(range(1, n_show+1))
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('scree_plot.png', dpi=150, bbox_inches='tight')
plt.show()

# Kaiser-Kriterium
n_kaiser = np.sum(eigenvalues > 1)
print(f"\nKaiser-Kriterium: {n_kaiser} Komponenten haben Eigenwert > 1")
print(f"Diese erklären {cumulative_variance[n_kaiser-1]*100:.1f}% der Varianz")

In [ ]:
# Detailanalyse: Wieviele Faktoren sind sinnvoll?
print("Analyse der Faktorenzahl:")
print("=" * 60)

# Kaiser-Kriterium
print(f"\n1. Kaiser-Kriterium (Eigenwert > 1): {n_kaiser} Faktoren")

# Elbow-Methode (manuell identifizieren)
eigenvalue_diffs = np.diff(eigenvalues[:10])
print(f"\n2. Eigenwert-Differenzen (für Elbow):")
for i, diff in enumerate(eigenvalue_diffs):
    print(f"   PC{i+1} → PC{i+2}: {diff:.3f}")

# 70% und 80% Varianz
n_70 = np.argmax(cumulative_variance >= 0.70) + 1
n_80 = np.argmax(cumulative_variance >= 0.80) + 1
print(f"\n3. Für 70% erklärte Varianz: {n_70} Faktoren")
print(f"   Für 80% erklärte Varianz: {n_80} Faktoren")

# Empfehlung basierend auf Literatur
print(f"\n4. Literatur (Hermann & Leuthold): 2-3 Faktoren")
print(f"   - 2 ausgeprägte Dimensionen")
print(f"   - 1 'halbe' Dimension (weniger stabil)")

## 5. PCA mit 2 Faktoren

In [ ]:
# PCA mit 2 Komponenten
pca_2 = PCA(n_components=2)
scores_2 = pca_2.fit_transform(X_scaled)

print(f"PCA mit 2 Faktoren:")
print(f"  Erklärte Varianz PC1: {pca_2.explained_variance_ratio_[0]*100:.2f}%")
print(f"  Erklärte Varianz PC2: {pca_2.explained_variance_ratio_[1]*100:.2f}%")
print(f"  Total: {pca_2.explained_variance_ratio_.sum()*100:.2f}%")

# Scores DataFrame
df_scores_2 = pd.DataFrame(
    scores_2,
    columns=['PC1', 'PC2'],
    index=df_imputed.index
).reset_index()

df_scores_2.head()

In [ ]:
# Visualisierung: Gemeinden im 2D-Raum
fig, ax = plt.subplots(figsize=(12, 10))

scatter = ax.scatter(
    df_scores_2['PC1'], 
    df_scores_2['PC2'],
    alpha=0.5,
    s=20,
    c='steelblue'
)

ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)

ax.set_xlabel(f'PC1 ({pca_2.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
ax.set_ylabel(f'PC2 ({pca_2.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
ax.set_title('Schweizer Gemeinden im politischen Raum (2 Dimensionen)', fontsize=14)

plt.tight_layout()
plt.savefig('pca_2d_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Loadings für 2-Faktor-Lösung
loadings_2 = pd.DataFrame(
    pca_2.components_.T,
    columns=['PC1', 'PC2'],
    index=df_imputed.columns
)

# Vorlage-Titel hinzufügen
proposal_titles = pd.read_sql_query(
    "SELECT proposal_id, title_de, voting_date FROM proposals p JOIN votings v ON p.voting_id = v.voting_id",
    conn
)
proposal_titles = proposal_titles.set_index('proposal_id')

loadings_2['title'] = loadings_2.index.map(lambda x: proposal_titles.loc[x, 'title_de'] if x in proposal_titles.index else 'Unknown')
loadings_2['date'] = loadings_2.index.map(lambda x: proposal_titles.loc[x, 'voting_date'] if x in proposal_titles.index else 'Unknown')

print("Top 10 Vorlagen mit höchster positiver Ladung auf PC1 (Links-Rechts?):")
print(loadings_2.nlargest(10, 'PC1')[['PC1', 'title', 'date']])

print("\nTop 10 Vorlagen mit höchster negativer Ladung auf PC1:")
print(loadings_2.nsmallest(10, 'PC1')[['PC1', 'title', 'date']])

In [ ]:
print("Top 10 Vorlagen mit höchster positiver Ladung auf PC2 (Konservativ-Liberal?):")
print(loadings_2.nlargest(10, 'PC2')[['PC2', 'title', 'date']])

print("\nTop 10 Vorlagen mit höchster negativer Ladung auf PC2:")
print(loadings_2.nsmallest(10, 'PC2')[['PC2', 'title', 'date']])

## 6. PCA mit 3 Faktoren

In [ ]:
# PCA mit 3 Komponenten
pca_3 = PCA(n_components=3)
scores_3 = pca_3.fit_transform(X_scaled)

print(f"PCA mit 3 Faktoren:")
print(f"  Erklärte Varianz PC1: {pca_3.explained_variance_ratio_[0]*100:.2f}%")
print(f"  Erklärte Varianz PC2: {pca_3.explained_variance_ratio_[1]*100:.2f}%")
print(f"  Erklärte Varianz PC3: {pca_3.explained_variance_ratio_[2]*100:.2f}%")
print(f"  Total: {pca_3.explained_variance_ratio_.sum()*100:.2f}%")

# Scores DataFrame
df_scores_3 = pd.DataFrame(
    scores_3,
    columns=['PC1', 'PC2', 'PC3'],
    index=df_imputed.index
).reset_index()

df_scores_3.head()

In [ ]:
# Loadings für 3-Faktor-Lösung
loadings_3 = pd.DataFrame(
    pca_3.components_.T,
    columns=['PC1', 'PC2', 'PC3'],
    index=df_imputed.columns
)

loadings_3['title'] = loadings_3.index.map(lambda x: proposal_titles.loc[x, 'title_de'] if x in proposal_titles.index else 'Unknown')
loadings_3['date'] = loadings_3.index.map(lambda x: proposal_titles.loc[x, 'voting_date'] if x in proposal_titles.index else 'Unknown')

print("Top 10 Vorlagen mit höchster positiver Ladung auf PC3 (Technokratisch-Ökologisch?):")
print(loadings_3.nlargest(10, 'PC3')[['PC3', 'title', 'date']])

print("\nTop 10 Vorlagen mit höchster negativer Ladung auf PC3:")
print(loadings_3.nsmallest(10, 'PC3')[['PC3', 'title', 'date']])

In [ ]:
# 3D-Visualisierung
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(
    df_scores_3['PC1'],
    df_scores_3['PC2'],
    df_scores_3['PC3'],
    alpha=0.5,
    s=15,
    c='steelblue'
)

ax.set_xlabel(f'PC1: Liberal ← → Konservativ ({pca_3.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2: Populistisch ← → Establishment ({pca_3.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_zlabel(f'PC3: Technokr. ← → Ökolog. ({pca_3.explained_variance_ratio_[2]*100:.1f}%)')
ax.set_title('Schweizer Gemeinden im politischen Raum (3 Dimensionen)', fontsize=14)

plt.tight_layout()
plt.savefig('pca_3d_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Paarweise 2D-Plots für alle drei Dimensionen
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# PC1 vs PC2
axes[0].scatter(df_scores_3['PC1'], df_scores_3['PC2'], alpha=0.4, s=15, c='steelblue')
axes[0].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[0].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[0].set_xlabel(f'PC1: Liberal ← → Konservativ ({pca_3.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2: Populistisch ← → Establishment ({pca_3.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('Gesellschaftlich vs Establishment')

# PC1 vs PC3
axes[1].scatter(df_scores_3['PC1'], df_scores_3['PC3'], alpha=0.4, s=15, c='steelblue')
axes[1].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[1].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[1].set_xlabel(f'PC1: Liberal ← → Konservativ ({pca_3.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC3: Technokr. ← → Ökolog. ({pca_3.explained_variance_ratio_[2]*100:.1f}%)')
axes[1].set_title('Gesellschaftlich vs Ökologie')

# PC2 vs PC3
axes[2].scatter(df_scores_3['PC2'], df_scores_3['PC3'], alpha=0.4, s=15, c='steelblue')
axes[2].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[2].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[2].set_xlabel(f'PC2: Populistisch ← → Establishment ({pca_3.explained_variance_ratio_[1]*100:.1f}%)')
axes[2].set_ylabel(f'PC3: Technokr. ← → Ökolog. ({pca_3.explained_variance_ratio_[2]*100:.1f}%)')
axes[2].set_title('Establishment vs Ökologie')

plt.tight_layout()
plt.savefig('pca_3_pairwise.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Interpretation der Achsen

In [ ]:
# Biplot für die ersten zwei Dimensionen (mit ausgewählten Vorlagen)
fig, ax = plt.subplots(figsize=(14, 12))

# Scores plotten (Gemeinden)
ax.scatter(df_scores_2['PC1'], df_scores_2['PC2'], alpha=0.3, s=10, c='lightgray', label='Gemeinden')

# Loadings als Vektoren (nur die extremsten)
n_arrows = 15

# Kombinierte Extremwerte finden
loadings_2['importance'] = np.sqrt(loadings_2['PC1']**2 + loadings_2['PC2']**2)
top_loadings = loadings_2.nlargest(n_arrows, 'importance')

# Skalierungsfaktor für bessere Sichtbarkeit
scale = 15

for idx, row in top_loadings.iterrows():
    ax.arrow(0, 0, row['PC1']*scale, row['PC2']*scale,
             head_width=0.3, head_length=0.2, fc='red', ec='red', alpha=0.7)
    # Kurztitel für Lesbarkeit
    short_title = row['title'][:40] + '...' if len(row['title']) > 40 else row['title']
    ax.annotate(short_title, 
                (row['PC1']*scale*1.1, row['PC2']*scale*1.1),
                fontsize=8, alpha=0.8)

ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
ax.set_xlabel(f'PC1 ({pca_2.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
ax.set_ylabel(f'PC2 ({pca_2.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
ax.set_title('Biplot: Gemeinden und einflussreichste Abstimmungsvorlagen', fontsize=14)

plt.tight_layout()
plt.savefig('biplot_2d.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Achseninterpretation basierend auf Ladungen
print("="*80)
print("INTERPRETATION DER POLITISCHEN DIMENSIONEN")
print("="*80)

print("\n" + "-"*80)
print("PC1: LIBERAL ↔ KONSERVATIV (gesellschaftliche Dimension)")
print("-"*80)
print("\nLIBERALER Pol (hohe PC1) - Städte wie Zürich, Genf, Lausanne:")
for idx, row in loadings_3.nlargest(5, 'PC1').iterrows():
    print(f"  {row['PC1']:+.3f}  {row['date']} - {row['title'][:60]}")

print("\nKONSERVATIVER Pol (tiefe PC1) - Innerschweiz wie Muotathal:")
for idx, row in loadings_3.nsmallest(5, 'PC1').iterrows():
    print(f"  {row['PC1']:+.3f}  {row['date']} - {row['title'][:60]}")

print("\n" + "-"*80)
print("PC2: ESTABLISHMENT ↔ POPULISTISCH")
print("-"*80)
print("\nESTABLISHMENT Pol (hohe PC2) - Goldküste folgt Bundesrat:")
for idx, row in loadings_3.nlargest(5, 'PC2').iterrows():
    print(f"  {row['PC2']:+.3f}  {row['date']} - {row['title'][:60]}")

print("\nPOPULISTISCHER Pol (tiefe PC2) - gegen Regierungsvorlagen:")
for idx, row in loadings_3.nsmallest(5, 'PC2').iterrows():
    print(f"  {row['PC2']:+.3f}  {row['date']} - {row['title'][:60]}")

print("\n" + "-"*80)
print("PC3: ÖKOLOGISCH ↔ TECHNOKRATISCH")
print("-"*80)
print("\nÖKOLOGISCHER Pol (hohe PC3) - Städte:")
for idx, row in loadings_3.nlargest(5, 'PC3').iterrows():
    print(f"  {row['PC3']:+.3f}  {row['date']} - {row['title'][:60]}")

print("\nTECHNOKRATISCHER Pol (tiefe PC3):")
for idx, row in loadings_3.nsmallest(5, 'PC3').iterrows():
    print(f"  {row['PC3']:+.3f}  {row['date']} - {row['title'][:60]}")

In [ ]:
# Extreme Gemeinden identifizieren
print("="*80)
print("EXTREME GEMEINDEN IN DEN POLITISCHEN DIMENSIONEN")
print("="*80)

print("\n" + "-"*60)
print("PC1: Am LIBERALSTEN (gesellschaftlich offen):")
print("-"*60)
for idx, row in df_scores_3.nlargest(5, 'PC1').iterrows():
    print(f"  {row['PC1']:+.2f}  {row['municipality_name']}")

print("\nPC1: Am KONSERVATIVSTEN (gesellschaftlich traditionell):")
for idx, row in df_scores_3.nsmallest(5, 'PC1').iterrows():
    print(f"  {row['PC1']:+.2f}  {row['municipality_name']}")

print("\n" + "-"*60)
print("PC2: Am meisten ESTABLISHMENT (folgt Bundesrat):")
print("-"*60)
for idx, row in df_scores_3.nlargest(5, 'PC2').iterrows():
    print(f"  {row['PC2']:+.2f}  {row['municipality_name']}")

print("\nPC2: Am meisten POPULISTISCH (gegen Regierungsvorlagen):")
for idx, row in df_scores_3.nsmallest(5, 'PC2').iterrows():
    print(f"  {row['PC2']:+.2f}  {row['municipality_name']}")

print("\n" + "-"*60)
print("PC3: Am ÖKOLOGISCHSTEN:")
print("-"*60)
for idx, row in df_scores_3.nlargest(5, 'PC3').iterrows():
    print(f"  {row['PC3']:+.2f}  {row['municipality_name']}")

print("\nPC3: Am TECHNOKRATISCHSTEN:")
for idx, row in df_scores_3.nsmallest(5, 'PC3').iterrows():
    print(f"  {row['PC3']:+.2f}  {row['municipality_name']}")

# Bekannte Referenzgemeinden
print("\n" + "="*80)
print("REFERENZGEMEINDEN ZUR ORIENTIERUNG")
print("="*80)
ref_cities = ['Zürich', 'Zumikon', 'Muotathal', 'Genève', 'Bern', 'Schwyz']
for city in ref_cities:
    match = df_scores_3[df_scores_3['municipality_name'].str.contains(city, case=False, na=False)]
    if len(match) > 0:
        r = match.iloc[0]
        lib = "liberal" if r['PC1'] > 0 else "konservativ"
        est = "establishment" if r['PC2'] > 0 else "populistisch"
        oeko = "ökolog." if r['PC3'] > 0 else "technokr."
        print(f"  {r['municipality_name']:<20} PC1={r['PC1']:+6.1f} PC2={r['PC2']:+6.1f} PC3={r['PC3']:+6.1f}  → {lib}, {est}, {oeko}")

## 8. Faktorrotation (Varimax) für bessere Interpretierbarkeit

In [ ]:
from scipy.stats import special_ortho_group
from scipy.linalg import svd

def varimax_rotation(loadings, max_iter=100, tol=1e-6):
    """
    Varimax-Rotation für bessere Interpretierbarkeit der Faktoren.
    """
    n_vars, n_factors = loadings.shape
    rotation_matrix = np.eye(n_factors)
    
    for _ in range(max_iter):
        old_rotation = rotation_matrix.copy()
        
        # Rotierte Ladungen
        rotated = loadings @ rotation_matrix
        
        # Varimax-Kriterium maximieren
        tmp = rotated ** 3 - rotated * (rotated ** 2).sum(axis=0) / n_vars
        u, s, vh = svd(loadings.T @ tmp)
        rotation_matrix = u @ vh
        
        # Konvergenzprüfung
        if np.max(np.abs(rotation_matrix - old_rotation)) < tol:
            break
    
    return loadings @ rotation_matrix, rotation_matrix

# Varimax auf 3-Faktor-Lösung anwenden
loadings_raw = pca_3.components_.T
loadings_rotated, rotation_matrix = varimax_rotation(loadings_raw)

print("Varimax-Rotation durchgeführt.")
print(f"\nRotationsmatrix:\n{rotation_matrix}")

In [ ]:
# Rotierte Ladungen analysieren
loadings_rot_df = pd.DataFrame(
    loadings_rotated,
    columns=['RC1', 'RC2', 'RC3'],
    index=df_imputed.columns
)

loadings_rot_df['title'] = loadings_rot_df.index.map(lambda x: proposal_titles.loc[x, 'title_de'] if x in proposal_titles.index else 'Unknown')
loadings_rot_df['date'] = loadings_rot_df.index.map(lambda x: proposal_titles.loc[x, 'voting_date'] if x in proposal_titles.index else 'Unknown')

print("="*80)
print("ROTIERTE FAKTOREN (VARIMAX)")
print("="*80)

for rc in ['RC1', 'RC2', 'RC3']:
    print(f"\n{'-'*60}")
    print(f"{rc}: Top 5 positive Ladungen")
    print(f"{'-'*60}")
    for idx, row in loadings_rot_df.nlargest(5, rc).iterrows():
        print(f"  {row[rc]:.3f}  {row['date']} - {row['title'][:55]}")
    
    print(f"\n{rc}: Top 5 negative Ladungen")
    for idx, row in loadings_rot_df.nsmallest(5, rc).iterrows():
        print(f"  {row[rc]:.3f}  {row['date']} - {row['title'][:55]}")

In [ ]:
# Rotierte Scores berechnen
scores_rotated = scores_3 @ rotation_matrix

df_scores_rot = pd.DataFrame(
    scores_rotated,
    columns=['RC1', 'RC2', 'RC3'],
    index=df_imputed.index
).reset_index()

# Paarweise Plots der rotierten Faktoren
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RC1 vs RC2
axes[0].scatter(df_scores_rot['RC1'], df_scores_rot['RC2'], alpha=0.4, s=15, c='steelblue')
axes[0].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[0].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[0].set_xlabel('RC1')
axes[0].set_ylabel('RC2')
axes[0].set_title('RC1 vs RC2 (Varimax-rotiert)')

# RC1 vs RC3
axes[1].scatter(df_scores_rot['RC1'], df_scores_rot['RC3'], alpha=0.4, s=15, c='steelblue')
axes[1].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[1].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('RC1')
axes[1].set_ylabel('RC3')
axes[1].set_title('RC1 vs RC3 (Varimax-rotiert)')

# RC2 vs RC3
axes[2].scatter(df_scores_rot['RC2'], df_scores_rot['RC3'], alpha=0.4, s=15, c='steelblue')
axes[2].axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
axes[2].axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
axes[2].set_xlabel('RC2')
axes[2].set_ylabel('RC3')
axes[2].set_title('RC2 vs RC3 (Varimax-rotiert)')

plt.tight_layout()
plt.savefig('pca_3_varimax_rotated.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Export der Ergebnisse

In [ ]:
# Scores exportieren für weitere Analysen
df_scores_3.to_csv('municipality_pca_scores_3factors.csv', index=False)
df_scores_rot.to_csv('municipality_pca_scores_3factors_rotated.csv', index=False)

# Loadings exportieren
loadings_3.to_csv('proposal_loadings_3factors.csv')
loadings_rot_df.to_csv('proposal_loadings_3factors_rotated.csv')

print("Ergebnisse exportiert:")
print("  - municipality_pca_scores_3factors.csv")
print("  - municipality_pca_scores_3factors_rotated.csv")
print("  - proposal_loadings_3factors.csv")
print("  - proposal_loadings_3factors_rotated.csv")

## 10. Zusammenfassung

In [ ]:
print("="*80)
print("ZUSAMMENFASSUNG DER PCA-ANALYSE")
print("="*80)

print(f"\nDatenbasis:")
print(f"  - {df_imputed.shape[0]} Gemeinden")
print(f"  - {df_imputed.shape[1]} Abstimmungsvorlagen")
print(f"  - Zeitraum: 2000-2025")

print(f"\nFaktorenanalyse:")
print(f"  - Kaiser-Kriterium empfiehlt: {n_kaiser} Faktoren")
print(f"  - Literatur empfiehlt: 2-3 Faktoren")

print(f"\nErklärte Varianz (3 Faktoren):")
print(f"  - PC1: {pca_3.explained_variance_ratio_[0]*100:.1f}%")
print(f"  - PC2: {pca_3.explained_variance_ratio_[1]*100:.1f}%")
print(f"  - PC3: {pca_3.explained_variance_ratio_[2]*100:.1f}%")
print(f"  - Total: {pca_3.explained_variance_ratio_.sum()*100:.1f}%")

print(f"\nInterpretation der Dimensionen:")
print(f"  - PC1: Liberal ↔ Konservativ (gesellschaftlich)")
print(f"         Vorlagen: Einbürgerung, EU, Ausländerpolitik")
print(f"         Liberal: Zürich, Genf, Lausanne | Konservativ: Muotathal, Unteriberg")
print(f"")
print(f"  - PC2: Establishment ↔ Populistisch")
print(f"         Vorlagen: AHV-Reformen, Regierungsvorlagen vs. populistische Initiativen")
print(f"         Establishment: Zumikon, Küsnacht | Populistisch: Innerschweiz, Berner Jura")
print(f"         HINWEIS: Nicht identisch mit Links-Rechts!")
print(f"")
print(f"  - PC3: Ökologisch ↔ Technokratisch")
print(f"         Vorlagen: Umwelt, Energie, Pestizide")
print(f"         Ökologisch: Bern, Basel, Zürich | Technokratisch: ländliche Gemeinden")

print(f"\nBefund:")
print(f"  Die Analyse findet 2 starke + 1 schwächere Dimension.")
print(f"  Die wirtschaftliche Links-Rechts-Dimension ist in PC2 mit")
print(f"  Establishment/Populismus-Aspekten vermischt.")

In [ ]:
# Datenbankverbindung schliessen
conn.close()
print("Datenbankverbindung geschlossen.")